In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install -e .

In [8]:
import timm
from PIL import Image
import urllib
import torch
from torch import nn
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

from src.data.project1.dataloader import get_loaders 

from src.utils import set_seed

#pytorch-lightning on top of PyTorch framework
import pytorch_lightning as pl
from torchmetrics.classification import BinaryAccuracy
accuracy = BinaryAccuracy()
from pytorch_lightning.callbacks import ModelCheckpoint 

In [9]:
set_seed(42)

## Loading model

In [2]:
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=2)

## Dataloader

In [50]:
loaders = get_loaders('/dtu/datasets1/02514/hotdog_nothotdog', batch_size=64, seed=0)

Computing std. dev. of training split...: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 1638/1638 [00:06<00:00, 238.67it/s]


Mean: tensor([0.5132, 0.4369, 0.3576])
Std. dev.: tensor([0.0214, 0.0208, 0.0223])


## Output of model

In [20]:
# Get example batch
batch, targets = next(iter(loaders['train'])) 
out = model(batch)
torch.argmax(nn.functional.softmax(out, dim=1), dim = 1)

tensor(1)

In [78]:
torch.nn.functional.one_hot(targets, num_classes=2)


tensor([[1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        

## Model-class (Lightning)

In [ ]:
class HotDogEfficientNet(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.criterion = nn.BCEWithLogitsLoss() # nn.BCELoss()
        self.efficient_net = timm.create_model('efficientnet_b0', pretrained=True, num_classes=2)

    def forward(self,x):

        out = self.efficient_net(x)

        return out # torch.argmax(nn.functional.softmax(out, dim=1), dim = 1)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr = 1e-4)

    def training_step(self,batch,batch_idx):
        x,y = batch
        y = torch.nn.functional.one_hot(y, num_classes=2) 

        y_hat = self(x)
        loss = self.criterion(y_hat,y)
        # logs metrics for each training_step - [default:True],
        # the average across the epoch, to the progress bar and logger-[default:False]
        acc = accuracy(y_hat,y)
        self.log("train_acc",acc,on_step=False,on_epoch=True,prog_bar=True,logger=True),
        self.log("train_loss",loss,on_step=False,on_epoch=True,prog_bar=True,logger=True)
        return loss

    def validation_step(self,batch,batch_idx):
        x,y = batch
        y = torch.nn.functional.one_hot(y, num_classes=2) 
        y_hat = self(x)
        loss = self.criterion(y_hat,y)
        acc = accuracy(y_hat,y)
        # logs metrics for each validation_step - [default:False]
        #the average across the epoch - [default:True]
        self.log("val_acc",acc,prog_bar=True,logger=True),
        self.log("val_loss",loss,prog_bar=True,logger=True)

# Defining model checkpoint

In [83]:
model_checkpoint = ModelCheckpoint(monitor = "val_loss",
                                   verbose=True,
                                   filename="{epoch}_{val_loss:.4f}")

## Training

In [84]:
# dm = HotDogDataModule()
hotdogModel = HotDogEfficientNet()

#CPU:default,GPU:gpus,TPU:tpu_cores
trainer = pl.Trainer(devices=1, accelerator="gpu", callbacks=[model_checkpoint]) 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [85]:
trainer.fit(model=hotdogModel,
            train_dataloaders = loaders['train'],
            val_dataloaders = loaders['validation']) 

#manually you can save best checkpoints - 
trainer.save_checkpoint("hotdog_efficient_net.ckpt")

/work3/s184984/venv/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:70: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type              | Params
----------------------------------------------------
0 | criterion     | BCEWithLogitsLoss | 0     
1 | efficient_net | EfficientNet      | 4.0 M 
----------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.040    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/work3/s184984/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/work3/s184984/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


RuntimeError: result type Float can't be cast to the desired output type Long